In [1]:
import BimatrixGame as BG
import globals as gl
import torch
import numpy as np
from environmentModelBase import Model, MixedStrategy, Strategy, StrategyType
import environmentModelBase as em
from learningBase import ReinforceAlgorithm
from neuralNetworkSimple import NNBase


In [2]:
np.random.seed(0)
gl.initialize()

In [3]:
number_rounds=50

# l1=NNBase(lr=gl.lr, num_input=gl.totalStages+2+gl.adversaryHistroy,num_actions=gl.numActions,adv_hist=gl.adversaryHistroy,action_step=gl.actionStep)
# l1.load("low,1684212411")
# l2=NNBase(lr=gl.lr, num_input=gl.totalStages+2+gl.adversaryHistroy,num_actions=gl.numActions,adv_hist=gl.adversaryHistroy,action_step=gl.actionStep)
# l2.load("low,1684332617")
# h1=NNBase(lr=gl.lr, num_input=gl.totalStages+2+gl.adversaryHistroy,num_actions=gl.numActions,adv_hist=gl.adversaryHistroy,action_step=gl.actionStep)
# h1.load("high,1684261807")
equilibria = []
low_cost_players = [
                    Strategy(StrategyType.static, NNorFunc=em.myopic, name="myopic"), 
                    Strategy(StrategyType.static, NNorFunc=em.const, name="const", firstPrice=132), 
                    Strategy(StrategyType.static, NNorFunc=em.guess, name="guess", firstPrice=132)
                   ]
high_cost_players = [
                    Strategy(StrategyType.static, NNorFunc=em.myopic, name="myopic"), 
                    Strategy(StrategyType.static, NNorFunc=em.const, name="const", firstPrice=132), 
                    Strategy(StrategyType.static, NNorFunc=em.guess, name="guess", firstPrice=132)
                   ]
bimatrixGame = BG.BimatrixGame(low_cost_players, high_cost_players)
    # bimatrixGame.reset_matrix()
bimatrixGame.fill_matrix()
low_cost_probabilities, high_cost_probabilities, low_cost_payoff, high_cost_payoff = bimatrixGame.compute_equilibria()
for round in range(number_rounds):
    print("Round", round, " of ", number_rounds)

    update = False


    acceptable, agentPayoffs, advPayoffs, low_cost_player = BG.training([gl.lowCost, gl.highCost], advMixedStrategy=MixedStrategy(
        strategiesList=high_cost_players, probablitiesArray=high_cost_probabilities), targetPayoff=low_cost_payoff)
    if acceptable:
        update = True
        low_cost_players.append(low_cost_player)
        bimatrixGame.add_low_cost_row(agentPayoffs, advPayoffs)
        equilibria.append(
            [low_cost_probabilities, high_cost_probabilities, low_cost_payoff, high_cost_payoff])
        print(f"low cost player {low_cost_player._name} added")

        low_cost_probabilities, high_cost_probabilities, low_cost_payoff, high_cost_payoff = bimatrixGame.compute_equilibria()


    acceptable, agentPayoffs, advPayoffs, high_cost_player = BG.training(
        [gl.highCost, gl.lowCost], advMixedStrategy=MixedStrategy(probablitiesArray=low_cost_probabilities, strategiesList=low_cost_players), targetPayoff=high_cost_payoff)

    if acceptable:
        update = True
        high_cost_players.append(high_cost_player)
        bimatrixGame.add_high_cost_col(advPayoffs, agentPayoffs)
        equilibria.append(
            [low_cost_probabilities, high_cost_probabilities, low_cost_payoff, high_cost_payoff])
        print(f"high cost player {high_cost_player._name} added")

        low_cost_probabilities, high_cost_probabilities, low_cost_payoff, high_cost_payoff = bimatrixGame.compute_equilibria()

    if update:
        gl.numEpisodes = gl.numEpisodesReset
    else:
        gl.numEpisodes += 1000

C:\Users\sjaha\Documents\EquiLearn\PGM_base\environmentModelBase.py:344: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  adversaryDist = Categorical(torch.tensor(self._strategyProbs))


equi: [0.00, 0.00, 1.00], [1.00, 0.00, 0.00], 154997.58, 79665.90
Round 0  of  50
1684386202 is trained against myopic-1.00,
low,1684386202 saved
low cost player low,1684386202 added
equi: [0.00, 0.00, 0.00, 1.00], [1.00, 0.00, 0.00], 178432.78, 35256.60
1684424924 is trained against low,1684386202-1.00,
high,1684424924 saved
high cost player high,1684424924 added
equi: [0.00, 0.00, 0.49, 0.51], [0.06, 0.00, 0.00, 0.94], 125951.71, 57157.89
Round 1  of  50
1684484716 is trained against myopic-0.06,high,1684424924-0.94,
low,1684484716 saved
low cost player low,1684484716 added
equi: [0.00, 0.00, 0.86, 0.00, 0.14], [0.61, 0.00, 0.39, 0.00], 149731.40, 74148.84
1684520477 is trained against guess-0.86,low,1684484716-0.14,
Round 2  of  50
1684557152 is trained against myopic-0.61,guess-0.39,
low,1684557152 saved
low cost player low,1684557152 added
equi: [0.00, 0.00, 0.00, 0.00, 1.00, 0.00], [0.00, 0.00, 0.00, 1.00], 125382.90, 51182.05
1684604536 is trained against low,1684484716-1.00,
Ro

KeyboardInterrupt: 

In [4]:
for strategy in bimatrixGame._strategies_low:
    print(strategy._name)

myopic
const
guess
low,1684386202
low,1684484716
low,1684557152
low,1684856358
low,1685028503


In [5]:
for strategy in bimatrixGame._strategies_high:
    print(strategy._name)

myopic
const
guess
high,1684424924
high,1684821735


In [12]:
sum=0
for i in range(26):
     sum+=3000*(i**1.2)

In [13]:
sum

1694226.0901287887

In [5]:
sum2=0
for i in range(26):
    sum2+=4000*i
sum2

1300000